#### A

In [7]:
import torch
from torch import nn
from torchvision import datasets, transforms, models

#### B

In [8]:
# 데이터 변환 (3채널로 확장 및 정규화)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # 1채널 -> 3채널로 변환
    transforms.Resize((224, 224)),               # 입력 크기를 모델에 맞게 변경
    transforms.ToTensor(),                       # 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화
])

# MNIST 데이터셋 로드
train_dataset = datasets.MNIST(root='MNIST_data/', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='MNIST_data/', train=False, download=True, transform=transform)

# DataLoader 설정
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

#### C

In [9]:
# 사전 학습된 ResNet18 로드
model = models.resnet18(pretrained=False)

# 마지막 레이어 수정 (10개의 MNIST 클래스에 맞게)
model.fc = nn.Linear(model.fc.in_features, 10)

# 모델을 GPU로 이동 (가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

#### D

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#### E

In [ ]:
# 학습 루프
epochs = 5
for epoch in range(epochs):
    model.train()  # 모델을 학습 모드로 전환
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 옵티마이저 초기화
        optimizer.zero_grad()

        # 예측 및 손실 계산
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 역전파 및 최적화
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")